# Iceberg with SQL Catalog

Example of Iceberg with SQL Postgres Catalog and Minio as S3 Compatible Storage.

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.master("spark://spark-master:7077")  # type: ignore
    .appName("Testing Iceberg with SQL Catalog")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .getOrCreate()
)

print(f'The PySpark {spark.version} version is running...')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/22 13:37:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The PySpark 3.5.0 version is running...


### Test Spark is running

In [23]:
data = [("John", 28), ("Alice", 35), ("Bob", 42)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.createOrReplaceTempView("tmp_people")
df.show()

+-----+---+
| Name|Age|
+-----+---+
| John| 28|
|Alice| 35|
|  Bob| 42|
+-----+---+



## Config Iceberg catalog

```bash
spark-sql \
    --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions \
    --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog \
    --conf spark.sql.catalog.spark_catalog.catalog-impl=org.apache.iceberg.jdbc.JdbcCatalog \
    --conf spark.sql.catalog.spark_catalog.jdbc.user=iceberg \
    --conf spark.sql.catalog.spark_catalog.jdbc.password=iceberg \
    --conf spark.sql.catalog.spark_catalog.uri=jdbc:postgresql://catalog:5432/iceberg \
    --conf spark.sql.catalog.spark_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO \
    --conf spark.sql.catalog.spark_catalog.s3.endpoint=http://minio:9000 \
    --conf spark.sql.catalog.spark_catalog.warehouse=s3://warehouse/
```

In [4]:
CATALOG_NAME = "demo"
DEFAULT_NAMESPACE = "default"

spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.default-namespace", DEFAULT_NAMESPACE)
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.uri", "jdbc:postgresql://catalog:5432/iceberg")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.jdbc.user", "iceberg")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.jdbc.password", "iceberg")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.s3.endpoint", "http://minio:9000")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.warehouse", "s3://warehouse/")

### Change Catalog

Activate the catalog and optionally the namespace.


In [5]:
%%sql
USE demo;

++
||
++
++

### Create new Namespace

In [45]:
%%sql
CREATE NAMESPACE new_namespace;

++
||
++
++

### Activate namespace

In [10]:
%%sql
USE demo.new_namespace;

++
||
++
++

### List existing catalogs, namespaces, tables

In [6]:
%%sql
SHOW catalogs;

catalog
demo
spark_catalog


In [15]:
%%sql
SHOW namespaces;

namespace
new
new_namespace


### Create table

In [16]:
%%sql
CREATE TABLE IF NOT EXISTS people (
  name STRING,
  age INT
) USING iceberg;

++
||
++
++

Create a table in a different Namespace than the one activated.

In [17]:
%%sql
CREATE TABLE IF NOT EXISTS default.quote (date Date, Close Float) USING iceberg PARTITIONED BY (YEAR(date));

++
||
++
++

### Show tables

In [20]:
%%sql
SHOW tables;

namespace,tableName,isTemporary
new_namespace,people,False


## Insert

In [21]:
quote = spark.read.csv("data/GDAXI.INDX.csv", header=True, inferSchema=True).select(["Date", "Close"])
quote.writeTo("default.quote").append()

In [24]:
%%sql
-- Insert from temp view
INSERT INTO people SELECT * FROM tmp_people

++
||
++
++

23/12/22 13:43:38 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/12/22 13:43:38 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

## Query Data

In [13]:
%%sql
SELECT * FROM people;

Py4JJavaError: An error occurred while calling o30.sql.
: org.apache.iceberg.exceptions.NotFoundException: Location does not exist: s3://warehouse/new_namespace/people/metadata/00005-b17537b3-620f-40f1-8ec2-3ca4f561d5bc.metadata.json
	at org.apache.iceberg.aws.s3.S3InputStream.openStream(S3InputStream.java:194)
	at org.apache.iceberg.aws.s3.S3InputStream.positionStream(S3InputStream.java:177)
	at org.apache.iceberg.aws.s3.S3InputStream.read(S3InputStream.java:107)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.ensureLoaded(ByteSourceJsonBootstrapper.java:539)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.detectEncoding(ByteSourceJsonBootstrapper.java:133)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.constructParser(ByteSourceJsonBootstrapper.java:256)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.JsonFactory._createParser(JsonFactory.java:1744)
	at

In [82]:
%%sql
SELECT * FROM quote;

date,Close
2010-01-04,6048.2998046875
2010-01-05,6031.85986328125
2010-01-06,6034.330078125
2010-01-07,6019.35986328125
2010-01-08,6037.60986328125
2010-01-11,6040.5
2010-01-12,5943.0
2010-01-13,5963.14013671875
2010-01-14,5988.8798828125
2010-01-15,5875.97021484375


### Stop Spark

In [ ]:
spark.stop()